In [1]:
from Data.data_loading import TilesDataset
from components.Eval_metrics import distance_metric
import pandas as pd 
from monai.utils import set_determinism
from components.Eval_metrics import score
from ModelBuilding.SegResNet_detection import SegResNet_Detection_Model, SegResNet_Detection_Config
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
import numpy as np
from tqdm import tqdm
import pandas as pd
import torchvision
import icecream
from icecream import ic
ic(torch.__version__)
ic(torchvision.__version__)
ic(torch.version.cuda)
ic(icecream.__version__)

SEED = 42
set_determinism(seed = SEED)




KeyboardInterrupt: 

In [ ]:
from torchvision.io import decode_image 
img = decode_image("/mnt/raid0/Kaggle/DS/byu-locating-bacterial-flagellar-motors-2025/train/tomo_00e463/slice_0000.jpg")
img.shape

In [ ]:
# def custom_collate(batch):
#     """
#     Custom collate function that handles variable-sized 3D volumes.
#     """
#     # Extract elements from the batch
#     images = [item['image'] for item in batch]  # List of tensors with different shapes
#     coords = [item['coords'] for item in batch]  # List of tensors
#     has_motor = torch.stack([item['has_motor'] for item in batch])  # Can be stacked as same shape
#     tomo_ids = [item['tomo_id'] for item in batch]  # List of strings/identifiers
#     voxel_spacings = torch.stack([item['voxel_spacing'] for item in batch])  # Can be stacked as same shape
    
#     # Return a dictionary with the batch data
#     return {
#         'image': images,  # List of tensors with different shapes
#         'coords': coords,  # List of tensors
#         'has_motor': has_motor,  # Tensor of shape [batch_size, 1]
#         'tomo_id': tomo_ids,  # List of strings/identifiers
#         'voxel_spacing': voxel_spacings  # Tensor of shape [batch_size, 1]
#     }   

In [ ]:
train_df = pd.read_csv('/mnt/raid0/Kaggle/DS/byu-locating-bacterial-flagellar-motors-2025/train_labels.csv')
train_img_paths = "/mnt/raid0/Kaggle/DS/byu-locating-bacterial-flagellar-motors-2025/train/"
ds = TilesDataset(train_df, train_img_paths)

In [ ]:
train_df = pd.read_csv('/mnt/raid0/Kaggle/DS/byu-locating-bacterial-flagellar-motors-2025/train_labels.csv')
train_img_paths = "/mnt/raid0/Kaggle/DS/byu-locating-bacterial-flagellar-motors-2025/train/"
# train_df = train_df[train_df['Number of motors'] <=1]  
#right now let us just focus on the cases with 1 or 0 motor. The Dataset class already supports multi label 
#we just need to create a multilabel loss, but not right now.

train_df = train_df.head(5)


full_dataset = TilesDataset(train_df, train_img_paths)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(
    full_dataset, 
    [train_size, val_size],
    generator=torch.Generator().manual_seed(SEED)
)

# Create data loaders
train_loader = DataLoader(
    train_dataset,
    batch_size=1,
    num_workers=0,
    pin_memory=True,
)
val_loader = DataLoader(
    val_dataset,
    batch_size=1,
    shuffle=False,
    num_workers=0,
    pin_memory=True,
    #collate_fn= custom_collate
)

config = SegResNet_Detection_Config()


In [ ]:
sample = train_dataset[2]


sample['image_tile'].shape

In [ ]:
# import pprint
# sample = train_loader.dataset[2]  # Get the 3rd item

# print("Image shape:", sample['image_tile'].shape)
# print("Coordinates:", sample['coords'])
# print("Has motor:", sample['has_motor'])
# print("Tomogram ID:", sample['tomo_id'])
# print("Voxel spacing:", sample['voxel_spacing'])
# pp = pprint.PrettyPrinter(indent=4)
# if isinstance(sample, dict):
# 	pp.pprint(sample.keys())  
# else:
# 	pp.pprint(dir(sample)) 



In [ ]:
sample = torch.randn(64,8,32,128,128)
sampel_two=  torch.randn(64,16,16,64,64)
sample = torch.flatten(sample,start_dim=1)

sample.shape
# features = torch.cat([sample, sampel_two])
# features.shape

In [ ]:
import torch
import os
import time
import gc
import pandas as pd
from tqdm import tqdm
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau

def memory_check():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    total_memory = torch.cuda.get_device_properties(device).total_memory
    allocated = torch.cuda.memory_allocated(device)
    reserved = torch.cuda.memory_reserved(device)

    print(f"Total memory: {total_memory / 1e9:.2f} GB")
    print(f"Allocated memory: {allocated / 1e9:.2f} GB")
    print(f"Reserved memory: {reserved / 1e9:.2f} GB")
def clear_gpu_memory():
    torch.cuda.empty_cache()
    gc.collect()

def save_checkpoint(model, optimizer, epoch, val_loss, val_fbeta, history, checkpoint_path):
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'val_loss': val_loss,
        'val_fbeta': val_fbeta,
        'history': history
    }, checkpoint_path)
    print(f"Checkpoint saved to {checkpoint_path}")

def train_model(
    model,
    train_loader,
    val_loader,
    optimizer,
    device,
    num_epochs=50,
    checkpoint_dir="checkpoints",
    scheduler=None,
    early_stopping_patience=10,
    min_radius=1000.0,
    beta=2.0,
    save_checkpoints=False 
):
    os.makedirs(checkpoint_dir, exist_ok=True)
    best_val_loss = float('inf')
    best_epoch = -1
    patience_counter = 0

    history = {
        'train_loss': [],
        'train_cls_loss': [],
        'train_coord_loss': [],
        'val_loss': [],
        'val_cls_loss': [],
        'val_coord_loss': [],
        'val_fbeta': []
    }

    print(f"Starting training on device: {device}")
    start_time = time.time()

    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        memory_check()
        model.train()
        train_loss = 0.0
        train_cls_loss = 0.0
        train_coord_loss = 0.0
        train_pbar = tqdm(train_loader, desc=f"Train Epoch {epoch+1}")
        for batch_data in train_pbar:
            volume = batch_data['image_tile'].to(device)
            has_motor = batch_data['has_motor'].to(device)
            coords = batch_data['local_coords'].to(device)
            voxel_spacing = batch_data['voxel_spacing'].to(device)
            optimizer.zero_grad()
            outputs = model(
                volume,
                labels={"has_motor": has_motor, "local_coords": coords}
            )
            loss = outputs["loss"]
            loss_dict = outputs["loss_dict"]
            loss.backward()
            optimizer.step()
            batch_loss = loss.item()
            batch_cls_loss = loss_dict["cls_loss"].item()
            batch_coord_loss = loss_dict["coord_loss"].item()
            train_loss += batch_loss
            train_cls_loss += batch_cls_loss
            train_coord_loss += batch_coord_loss
            train_pbar.set_postfix({
                'loss': f"{batch_loss:.4f}",
                'coord_loss': f"{batch_coord_loss:.4f}"
            })
            clear_gpu_memory()
        train_loss /= len(train_loader)
        train_cls_loss /= len(train_loader)
        train_coord_loss /= len(train_loader)

        model.eval()
        val_loss = 0.0
        val_cls_loss = 0.0
        val_coord_loss = 0.0
        val_predictions = []
        val_targets = []
        with torch.no_grad():
            val_pbar = tqdm(val_loader, desc=f"Validation Epoch {epoch+1}")
            for batch_data in val_pbar:
                volume = batch_data['image'].to(device)
                has_motor = batch_data['has_motor'].to(device)
                coords = batch_data['local_coords'].to(device)
                voxel_spacing = batch_data['voxel_spacing'].to(device)
                tomo_ids = batch_data['tomo_id']
                outputs = model(
                    volume,
                    labels={"has_motor": has_motor, "local_coords": coords}
                )
                loss = outputs["loss"]
                loss_dict = outputs["loss_dict"]
                batch_loss = loss.item()
                batch_cls_loss = loss_dict["cls_loss"].item()
                batch_coord_loss = loss_dict["coord_loss"].item()
                val_loss += batch_loss
                val_cls_loss += batch_cls_loss
                val_coord_loss += batch_coord_loss
                pred_has_motor = outputs["has_motor"].cpu().numpy()
                pred_coords = outputs["coordinates"].cpu().numpy()
                true_has_motor = has_motor.cpu().numpy()
                true_coords = coords.cpu().numpy()
                voxel_spacing_np = voxel_spacing.cpu().numpy()
                for i in range(len(tomo_ids)):
                    val_predictions.append({
                        'tomo_id': tomo_ids[i],
                        'Has motor': 1 if pred_has_motor[i] > 0.5 else 0,
                        'Motor axis 0': pred_coords[i, 0],
                        'Motor axis 1': pred_coords[i, 1],
                        'Motor axis 2': pred_coords[i, 2],
                    })
                    val_targets.append({
                        'tomo_id': tomo_ids[i],
                        'Has motor': true_has_motor[i, 0],
                        'Motor axis 0': true_coords[i, 0],
                        'Motor axis 1': true_coords[i, 1],
                        'Motor axis 2': true_coords[i, 2],
                        'Voxel spacing': voxel_spacing_np[i, 0]
                    })
                val_pbar.set_postfix({
                    'loss': f"{batch_loss:.4f}",
                    'cls_loss': f"{batch_cls_loss:.4f}",
                    'coord_loss': f"{batch_coord_loss:.4f}"
                })
        val_loss /= len(val_loader)
        val_cls_loss /= len(val_loader)
        val_coord_loss /= len(val_loader)
        val_pred_df = pd.DataFrame(val_predictions)
        val_target_df = pd.DataFrame(val_targets)
        val_fbeta = score(
            val_target_df,
            val_pred_df,
            min_radius=min_radius,
            beta=beta
        )
        if scheduler is not None:
            if hasattr(scheduler, 'step'):
                scheduler.step(val_loss)

        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"Train Loss: {train_loss:.4f}, CLS: {train_cls_loss:.4f}, COORD: {train_coord_loss:.4f}")
        print(f"Val Loss: {val_loss:.4f}, CLS: {val_cls_loss:.4f}, COORD: {val_coord_loss:.4f}, F-beta: {val_fbeta:.4f}")

        history['train_loss'].append(train_loss)
        history['train_cls_loss'].append(train_cls_loss)
        history['train_coord_loss'].append(train_coord_loss)
        history['val_loss'].append(val_loss)
        history['val_cls_loss'].append(val_cls_loss)
        history['val_coord_loss'].append(val_coord_loss)
        history['val_fbeta'].append(val_fbeta)

        # Early stopping and best model logic
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_epoch = epoch
            patience_counter = 0
            # Optionally save best model checkpoint
            if save_checkpoints:
                checkpoint_path = os.path.join(checkpoint_dir, "best_model.pt")
                save_checkpoint(model, optimizer, epoch, val_loss, val_fbeta, history, checkpoint_path)
        else:
            patience_counter += 1
            print(f"Validation loss didn't improve. Patience: {patience_counter}/{early_stopping_patience}")

        # Optionally save checkpoint every epoch
        if save_checkpoints:
            checkpoint_path = os.path.join(checkpoint_dir, f"model_epoch_{epoch+1}.pt")
            save_checkpoint(model, optimizer, epoch, val_loss, val_fbeta, history, checkpoint_path)

        if patience_counter >= early_stopping_patience:
            print(f"Early stopping triggered after {epoch+1} epochs")
            break

    time_elapsed = time.time() - start_time
    print(f"Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s")
    print(f"Best validation loss: {best_val_loss:.4f} at epoch {best_epoch+1}")
    
    if save_checkpoints and os.path.exists(os.path.join(checkpoint_dir, "best_model.pt")):
        checkpoint = torch.load(os.path.join(checkpoint_dir, "best_model.pt"))
        model.load_state_dict(checkpoint['model_state_dict'])

    clear_gpu_memory()
    return model, history



model = SegResNet_Detection_Model(config)

# Move model to device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Create optimizer and scheduler
optimizer = Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(
    optimizer, 
    mode='min', 
    factor=0.5, 
    patience=5, 
)

# Train the model
trained_model, history = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    device=device,
    num_epochs=50,
    checkpoint_dir="checkpoints",
    scheduler=scheduler,
    early_stopping_patience=10,
    min_radius=1000.0,
    beta=2.0
)


NameError: name 'config' is not defined

In [ ]:

model = SegResNet_Detection_Model(SegResNet_Detection_Config())

# Move model to device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [1]:
import torch
print(torch.__version__)

AttributeError: module 'torch' has no attribute '__version__'

In [ ]:
dummy = torch.randn([4,1,32,32,32]).to('cuda')
output =  model(dummy)
dummy.shape

ic| self.dropout(x).shape: torch.Size([4, 32, 32, 32, 32])
ic| 'Shape before down_layers'
ic| f"downlayer {layer} output shape:": 'downlayer 0 output shape:'
    x.shape: torch.Size([4, 32, 32, 32, 32])
ic| f"downlayer {layer} output shape:": 'downlayer 1 output shape:'
    x.shape: torch.Size([4, 64, 16, 16, 16])
ic| f"downlayer {layer} output shape:": 'downlayer 2 output shape:'
    x.shape: torch.Size([4, 128, 8, 8, 8])
ic| f"downlayer {layer} output shape:": 'downlayer 3 output shape:'
    x.shape: torch.Size([4, 256, 4, 4, 4])


Input shape: torch.Size([4, 1, 32, 32, 32])


torch.Size([4, 1, 32, 32, 32])

In [ ]:
for item in output.items():
    print(item)

('probabilities', tensor([[[[[0.0474, 0.0174, 0.0293,  ..., 0.0255, 0.0244, 0.0177],
           [0.0200, 0.0169, 0.0275,  ..., 0.0243, 0.0260, 0.0115],
           [0.0236, 0.0227, 0.0281,  ..., 0.0147, 0.0404, 0.0239],
           ...,
           [0.0138, 0.0126, 0.0673,  ..., 0.0675, 0.0674, 0.0519],
           [0.0200, 0.0131, 0.0319,  ..., 0.0147, 0.0596, 0.0339],
           [0.0364, 0.0137, 0.0239,  ..., 0.0387, 0.0559, 0.0314]],

          [[0.0279, 0.0184, 0.0348,  ..., 0.0138, 0.0159, 0.0193],
           [0.0192, 0.0247, 0.0153,  ..., 0.0079, 0.0087, 0.0152],
           [0.0080, 0.0030, 0.0126,  ..., 0.0092, 0.0272, 0.0121],
           ...,
           [0.0158, 0.0103, 0.0076,  ..., 0.0080, 0.0180, 0.0374],
           [0.0205, 0.0101, 0.0178,  ..., 0.0551, 0.0330, 0.0194],
           [0.0167, 0.0164, 0.0276,  ..., 0.0311, 0.0325, 0.0259]],

          [[0.0101, 0.0170, 0.0108,  ..., 0.0793, 0.0078, 0.0204],
           [0.0086, 0.0124, 0.0216,  ..., 0.0113, 0.0068, 0.0079],
        